In [2]:
import pandas as pd

In [3]:
data_full = pd.read_csv("C:/Users/rafal/Desktop/DS/Merceri price/train.tsv", sep = '\t')

In [4]:
data = data_full.sample(100000)

Używam tylko 100 000 obserwacji.

In [121]:
data.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
360135,360135,CHANEL SPF 12 FOUNDATION IN WALNUT,2,Beauty/Makeup/Face,Chanel,30.0,1,-too dark for me -used once -like second skin ...
405088,405088,T-shirts,2,Women/Tops & Blouses/T-Shirts,NaN,7.0,0,XL size Check out my other items to save on sh...
100509,100509,14K Amethyst Ring,1,Women/Jewelry/Rings,NaN,10.0,1,"Size 7 1/2. White gold filled, amethyst ring. ..."
543338,543338,karaoke singing machine,3,"Other/Musical instruments/DJ, Electronic Music...",Singing Machine,15.0,0,The singing machine doesn't come with micropho...
1286895,1286895,Adidas,3,Men/Shoes/Athletic,Adidas,12.0,0,Light weight Good condition/ lots of life left :)


In [28]:
pv = pd.pivot_table(data, values = ['price'],index = ['brand_name'], aggfunc = ['count', np.mean, np.median])

In [29]:
pv.columns = pv.columns.get_level_values(0)

In [37]:
pv_sorted = pv.sort_values(by = ['count'], ascending = False)

In [61]:
data.describe()

,train_id,item_condition_id,price,shipping
count,1.482535e+06,1.482535e+06,1.482535e+06,1.482535e+06
mean,7.412670e+05,1.907380e+00,2.673752e+01,4.472744e-01
std,4.279711e+05,9.031586e-01,3.858607e+01,4.972124e-01
min,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
25%,3.706335e+05,1.000000e+00,1.000000e+01,0.000000e+00
50%,7.412670e+05,2.000000e+00,1.700000e+01,0.000000e+00
75%,1.111900e+06,3.000000e+00,2.900000e+01,1.000000e+00
max,1.482534e+06,5.000000e+00,2.009000e+03,1.000000e+00


In [5]:
categories = data['category_name'].str.split('/', 3, expand = True)

In [6]:
categories.isna().sum()

0      418
1      418
2      418
3    99700
dtype: int64

Wyrzucam 4 poziom kategorii i uzupełniamy NA.

In [8]:
categories = categories.drop(columns = 3, axis = 1)
categories = categories.fillna(value = 'No_category')

In [9]:
categories.head()

,0,1,2
775838,Beauty,Fragrance,Women
984982,Other,Daily & Travel items,Health Care
1144303,Kids,Girls (4+),Coats & Jackets
1222558,Women,Sweaters,Crewneck
1110435,Women,Swimwear,One-Piece


In [10]:
categories.isna().sum()

0    0
1    0
2    0
dtype: int64

In [11]:
data_cats = pd.concat([data, categories], axis = 1)

In [13]:
data_cats.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,0,1,2
775838,775838,Chanel Coco Mademoiselle,1,Beauty/Fragrance/Women,Chanel,81.0,0,It's brand new and 100% authentic. This is a g...,Beauty,Fragrance,Women
984982,984982,Young living Raven,3,Other/Daily & Travel items/Health Care,NaN,13.0,0,Partially used as shown in pic,Other,Daily & Travel items,Health Care
1144303,1144303,Nwt Northface Reversible Jacket xs 6,1,Kids/Girls (4+)/Coats & Jackets,The North Face,56.0,0,Girls extra small xs 6 Northface jacket Super ...,Kids,Girls (4+),Coats & Jackets
1222558,1222558,Lauren Conrad knit sweater!,2,Women/Sweaters/Crewneck,LC Lauren Conrad,8.0,1,Like new! Light pink!,Women,Sweaters,Crewneck
1110435,1110435,ONE PIECE bathing suit,2,Women/Swimwear/One-Piece,NaN,17.0,0,This is just for the one piece! Size small/medium,Women,Swimwear,One-Piece


In [14]:
y = data_cats.loc[:, 'price']
data_features = data_cats.drop(columns = ['price', 'category_name'], axis = 1)

In [15]:
data_features.loc[:, 'brand_name'] = data_features.loc[:, 'brand_name'].fillna(value = 'No_brand_name')

In [16]:
data_features.head()

,train_id,name,item_condition_id,brand_name,shipping,item_description,0,1,2
775838,775838,Chanel Coco Mademoiselle,1,Chanel,0,It's brand new and 100% authentic. This is a g...,Beauty,Fragrance,Women
984982,984982,Young living Raven,3,No_brand_name,0,Partially used as shown in pic,Other,Daily & Travel items,Health Care
1144303,1144303,Nwt Northface Reversible Jacket xs 6,1,The North Face,0,Girls extra small xs 6 Northface jacket Super ...,Kids,Girls (4+),Coats & Jackets
1222558,1222558,Lauren Conrad knit sweater!,2,LC Lauren Conrad,1,Like new! Light pink!,Women,Sweaters,Crewneck
1110435,1110435,ONE PIECE bathing suit,2,No_brand_name,0,This is just for the one piece! Size small/medium,Women,Swimwear,One-Piece


## Preprocessing the name variable

In [1]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rafal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [44]:
corpus = []
for i in range(0, 100000):
    name = re.sub('[^a-zA-Z]', ' ', data_features.iloc[i, 1])
    name = name.lower()
    name = name.split()
    ps = PorterStemmer()
    name = [ps.stem(word) for word in name if not word in set(stopwords.words('english'))]
    name = ' '.join(name)
    corpus.append(name)

In [46]:
# Creating the Bag of Words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1000)
name_features = cv.fit_transform(corpus)

In [60]:
df_name_features = pd.DataFrame(name_features.toarray())

In [69]:
df_name_features.head()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [71]:
data_features.reset_index(drop = True, inplace = True)
data_c_name = pd.concat([data_features, df_name_features], axis = 1)

In [73]:
data_c_name.drop(columns = 'name', inplace=True)

In [74]:
data_c_name.head()

,train_id,item_condition_id,brand_name,shipping,item_description,0,1,2,0,1,...,990,991,992,993,994,995,996,997,998,999
0,775838,1,Chanel,0,It's brand new and 100% authentic. This is a g...,Beauty,Fragrance,Women,0,0,...,0,0,0,0,0,0,0,0,0,0
1,984982,3,No_brand_name,0,Partially used as shown in pic,Other,Daily & Travel items,Health Care,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1144303,1,The North Face,0,Girls extra small xs 6 Northface jacket Super ...,Kids,Girls (4+),Coats & Jackets,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1222558,2,LC Lauren Conrad,1,Like new! Light pink!,Women,Sweaters,Crewneck,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1110435,2,No_brand_name,0,This is just for the one piece! Size small/medium,Women,Swimwear,One-Piece,0,0,...,0,0,0,0,0,0,0,0,0,0


In [81]:
len(data_c_name['brand_name'].unique())

2037

In [89]:
brands = pd.pivot_table(data = data_cats, values = 'price', aggfunc= 'count', index = 'brand_name')

In [94]:
brands.sort_values(by = 'price', ascending=False, inplace=True)
brands.rename(columns = {'price':'count'}, inplace=True)

In [108]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [118]:
preprocess = ColumnTransformer([('brand_name', OneHotEncoder(), 'brand_name')])

In [126]:
ohe = OneHotEncoder(categories=brands.iloc[0:172, 0])

In [128]:
brand_names = ohe.fit_transform(data_c_name.iloc[:,2:3])

C:\Users\rafal\Anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
df4 = df3[df3.columns[df3.columns.isin(brands.iloc[0:172, 0])]]

In [114]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer

In [ ]:
preprocess = make_column_transformer